In [12]:
# Standard imports
import json
from tqdm import tqdm
import sys
import pandas as pd
import random
import os
from joblib import Parallel, delayed
import importlib
from functools import partial
import csv
import dotenv
dotenv.load_dotenv()

True

In [13]:
# Method imports
sys.path.append('..')

from methods.mathprompterpp import mathprompterpp
from methods.mathprompter import mathprompter
from methods.zeroshotCoT import zeroshotCoT
from utils.method_utils import generating_algebraic_template
from utils.plot_utils import compute_detailed_metrics, plot_individual_metrics
from llm_inference.llm_factory import get_llm_service

import methods.mathprompterpp
import methods.mathprompter
import methods.zeroshotCoT
import utils.method_utils
import llm_inference.llm_factory

importlib.reload(methods.mathprompterpp)
importlib.reload(methods.zeroshotCoT)   
importlib.reload(utils.method_utils)
importlib.reload(llm_inference.llm_factory)


<module 'llm_inference.llm_factory' from '/Users/s/Desktop/Academics/Quarter 4/CSE256/Project/FinalCode/llm_inference/llm_factory.py'>

In [14]:
# Initialize parameters
suffix = str(random.randint(1, 1000000))
random.seed(42)
# model_service = 'openai'
# model_name = 'gpt-4o-mini'
model_service = 'llama'
model_name = 'mlx-community/Llama-3.2-1B-Instruct-4bit'
data_size = 100
max_tokens = 200
few_shot = False
consensus_k = 5

In [15]:
# Initialize model and data
# SERVICE = get_llm_service(model_service, model_name, temperature=0.0, max_tokens=max_tokens)

In [16]:
# Define the methods you want to evaluate
run_methods = ["MathPrompter", "MathPrompter++", "Zeroshot CoT"]  # Example methods

In [17]:
with open('data/SVAMP.json', 'r') as file:
    data = json.load(file)
data = random.sample(data, k=data_size)  # Take a random sample of 100 rows

suffix += "_" + model_name.split("/")[-1] + "_" + str(data_size) 
suffix += "_few_shot" if few_shot else ""
print(suffix)

# Create folder if not exists
if not os.path.exists('results/' + suffix):
    os.makedirs('results/' + suffix)

472408_Llama-3.2-1B-Instruct-4bit_100


In [18]:
if "MathPrompter" in run_methods:
    # MathPrompter
    header_mathprompter = ["Question", "Type", "Correct Answer", "Predicted Answer", "Predicted Algebraic Answer", "Predicted Python Answer", "Generated Algebraic Template", "Generated Expression", "Generated Python Code", "Variable Mapping", "Error", "Error Message"]
    df_mathprompter = pd.DataFrame(columns=header_mathprompter)
    csv_file_mathprompter = 'results/' + suffix + '/mathprompter_results.csv'  # CSV file path

if "MathPrompter++" in run_methods:
    # MathPrompter++
    header_mathprompterpp = ["Question", "Type", "Correct Answer", "Predicted Answer", "Predicted Algebraic Answer", "Predicted Python Answer", "Generated Algebraic Template", "Rough Solution", "Generated Expression", "Generated Python Code", "Variable Mapping", "Error", "Error Message"]
    df_mathprompterpp = pd.DataFrame(columns=header_mathprompterpp)
    csv_file_mathprompterpp = 'results/' + suffix + '/mathprompterpp_results.csv'  # CSV file path

if "Zeroshot CoT" in run_methods:
    # Zeroshot CoT
    header_zeroshot_cot = ["Question", "Type", "Correct Answer", "Predicted Answer", "Response", "Error", "Error Message"]
    df_zeroshot_cot = pd.DataFrame(columns=header_zeroshot_cot)
    csv_file_zeroshot_cot = 'results/' + suffix + '/zeroshot_cot_results.csv'  # CSV file path

# Example of saving the DataFrames to CSV files
if "MathPrompter" in run_methods:
    df_mathprompter.to_csv(csv_file_mathprompter, index=False)

if "MathPrompter++" in run_methods:
    df_mathprompterpp.to_csv(csv_file_mathprompterpp, index=False)

if "Zeroshot CoT" in run_methods:
    df_zeroshot_cot.to_csv(csv_file_zeroshot_cot, index=False)


In [19]:
def reset_csv_files():
    if "MathPrompter" in run_methods:
        with open(csv_file_mathprompter, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header_mathprompter)
    if "MathPrompter++" in run_methods:
        with open(csv_file_mathprompterpp, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header_mathprompterpp)
    if "Zeroshot CoT" in run_methods:
        with open(csv_file_zeroshot_cot, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header_zeroshot_cot)


In [20]:
# Parallelize across different data rows using joblib
def process_question(item, debug = False):
    SERVICE = get_llm_service(model_service, model_name, temperature=0.0, max_tokens=max_tokens)
    question = item["Body"] + " " + item["Question"]
    correct_answer = item["Answer"]
    question_type = item["Type"]
    qt, variable_mapping = generating_algebraic_template(question) 
    if "MathPrompter" in run_methods:   
        mathprompter_answer = mathprompter(qt, variable_mapping, SERVICE, k = consensus_k, few_shot=few_shot)

        # Create a dictionary for the new row
        new_row = {
            "Question": question,
            "Type": question_type,
            "Correct Answer": correct_answer,
            "Predicted Answer": mathprompter_answer["Result"],
            "Predicted Algebraic Answer": mathprompter_answer["Expression_Result"],
            "Predicted Python Answer": mathprompter_answer["Code_Result"],
            "Generated Algebraic Template": qt,
            "Generated Expression": mathprompter_answer["Expression"],
            "Generated Python Code": mathprompter_answer["Code"],
            "Variable Mapping": variable_mapping,
            "Error": mathprompter_answer["Error"],
            "Error Message": mathprompter_answer["Error_Message"]
        }
        if debug:
            print("MathPrompter")
            print(mathprompter_answer)
            print(new_row)
        # Append the new row to the DataFrame using pd.concat
        # df_mathprompter = pd.concat([df_mathprompter, pd.DataFrame([new_row])], ignore_index=True)
        # Write to csv
        with open(csv_file_mathprompter, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_row.values())
    if "MathPrompter++" in run_methods:
        mathprompterpp_answer = mathprompterpp(question, qt, variable_mapping, SERVICE, k = consensus_k, few_shot=few_shot)
        
        # Create a dictionary for the new row
        new_row = {
            "Question": question,
            "Type": question_type,
            "Correct Answer": correct_answer,
            "Predicted Answer": mathprompterpp_answer["Result"],
            "Predicted Algebraic Answer": mathprompterpp_answer["Expression_Result"],
            "Predicted Python Answer": mathprompterpp_answer["Code_Result"],
            "Generated Algebraic Template": qt,
            "Rough Solution": mathprompterpp_answer["Rough_Solution"],
            "Generated Expression": mathprompterpp_answer["Expression"],
            "Generated Python Code": mathprompterpp_answer["Code"],
            "Variable Mapping": variable_mapping,
            "Error": mathprompterpp_answer["Error"],
            "Error Message": mathprompterpp_answer["Error_Message"]
        }
        if debug:
            print("MathPrompter++")
            print(mathprompterpp_answer)
            print(new_row)
        # Append the new row to the DataFrame using pd.concat
        # df_mathprompterpp = pd.concat([df_mathprompterpp, pd.DataFrame([new_row])], ignore_index=True)
        # Write to csv
        with open(csv_file_mathprompterpp, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_row.values())
    if "Zeroshot CoT" in run_methods:
        zeroshot_cot_answer = zeroshotCoT(question, SERVICE, max_tokens)    

        # Create a dictionary for the new row
        new_row = {
            "Question": question,
            "Type": question_type,
            "Correct Answer": correct_answer,
            "Predicted Answer": zeroshot_cot_answer["Answer"],
            "Response": zeroshot_cot_answer["Response"],
            "Error": zeroshot_cot_answer["Error"],
            "Error Message": zeroshot_cot_answer["Error_Message"]
        }
        if debug:
            print("Zeroshot CoT")
            print(zeroshot_cot_answer)
            print(new_row)
        # Append the new row to the DataFrame using pd.concat
        # df_zeroshot_cot = pd.concat([df_zeroshot_cot, pd.DataFrame([new_row])], ignore_index=True)
        # Write to csv
        with open(csv_file_zeroshot_cot, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_row.values())

In [21]:
# Test 
# reset_csv_files()
# # if not few_shot:
# #     run_methods = ["MathPrompter", "MathPrompter++"]
# for i in tqdm(range(100)):
#     # print(data[i])
#     process_question(data[i], debug=False)

In [22]:
# Run the parallelized function
reset_csv_files()
process_question_partial = partial(process_question, debug=False)
Parallel(n_jobs=6)(delayed(process_question_partial)(item) for item in tqdm(data[:100], desc="Processing Questions"))

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 32305.29it/s]


30
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
105
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
Syntax error in the expression: invalid syntax (<string>, line 1)
An error occurred during the verification process: must be real number, not NoneType


Processing Questions:   2%|▏         | 2/100 [00:10<00:05, 19.43it/s]

Syntax error in the expression: invalid syntax (<string>, line 1)
An error occurred during the verification process: must be real number, not NoneType


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 33644.15it/s]it]


...

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 19676.17it/s]


Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
5
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
Syntax error in the expression: invalid syntax (<string>, line 1)
An error occurred during the verification process: must be real number, not NoneType
Syntax error in the expression: invalid syntax (<string>, line 1)
An error occurred during the verification process: must be real number, not NoneType


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 16341.44it/s]


An error occurred during the verification process: 'Code_Result'


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [23]:
process_question(data[0], debug=True)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 115439.56it/s]


30
Error when executing Python code: name 'solution' is not defined
An error occurred during the verification process: must be real number, not NoneType
MathPrompter
defaultdict(<function mathprompter.<locals>.<lambda> at 0x30a50ab90>, {'Expression': 'var_A - var_B', 'Code': 'def find_bottle_caps(var_A, var_B):\n    return var_A + var_B\n\nprint(find_bottle_caps(10, 20))', 'Error': True, 'Error_Message': 'must be real number, not NoneType', 'Result': None, 'Expression_Result': None, 'Code_Result': None})
{'Question': 'Danny collects bottle caps. He lost 66 bottle caps at the park. Now he has 25 bottle caps in his collection. How many bottle caps did danny have at first?', 'Type': 'Addition', 'Correct Answer': 91.0, 'Predicted Answer': None, 'Predicted Algebraic Answer': None, 'Predicted Python Answer': None, 'Generated Algebraic Template': 'Danny collects bottle caps. He lost var_A bottle caps at the park. Now he has var_B bottle caps in his collection. How many bottle caps did danny h